In [1]:
import math
import numpy as np

#sin_wave = np.array([math.sin(x) for x in np.arange(200)])
sin_wave = np.arange(0,50)

sin_wave

X = []
Y = []

seq_len = 4
num_records = len(sin_wave) - seq_len

In [2]:
sin_wave

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [3]:

for i in range(num_records - 2):
    X.append(sin_wave[i:i+seq_len])
    Y.append(sin_wave[i+seq_len])

In [4]:
X[:5]

[array([0, 1, 2, 3]),
 array([1, 2, 3, 4]),
 array([2, 3, 4, 5]),
 array([3, 4, 5, 6]),
 array([4, 5, 6, 7])]

In [ ]:
'''
We are considering first 4 records as input and Next immediate record as output 
You can see the same below

'''

In [5]:
Y[:6]

[4, 5, 6, 7, 8, 9]

In [6]:
len(X)

44

In [7]:
# Convert the input data in teh format of [batchsize,timestep,input feaures]

X = np.array(X)
X = np.expand_dims(X, axis=2)

In [8]:
X.shape  # batches = 44; timestep 4; input features = 1


(44, 4, 1)

In [9]:
X[0]   # first sample

array([[0],
       [1],
       [2],
       [3]])

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [38]:
timestep = 4

hidden_dim = 8 # hidden unit
output_dim = 1

learning_rate = 0.0001

bptt_truncate = 5
min_clip_value = -10
max_clip_value = 10


U = np.random.uniform(0, 1, (hidden_dim, timestep))  # weights between input and hidden layers
V = np.random.uniform(0, 1, (output_dim, hidden_dim)) # weights between hidden and output layers
W = np.random.uniform(0, 1, (hidden_dim, hidden_dim)) # shared weights in the RNN layer (hidden layer)

In [39]:
Y = np.array(Y).reshape(len(Y),1)

In [40]:
Y.shape

(44, 1)

In [61]:
# Train the Model
loss = []

val_loss = 0.0
for epoch in range(10):
    
    for i in range(Y.shape[0]):

        x,y = X[i],Y[i]

        layers = []

        prev_s = np.zeros((hidden_dim,1))

        dU = np.zeros(U.shape)
        dV = np.zeros(V.shape)
        dW = np.zeros(W.shape)

        dU_t = np.zeros(U.shape)
        dV_t = np.zeros(V.shape)
        dW_t = np.zeros(W.shape)


        # forward pass
        for t in range(timestep):
            new_input = np.zeros(x.shape)
            new_input[t] = x[t]
            mulu = np.dot(U, new_input)
            mulw = np.dot(W, prev_s)
            add = mulw + mulu
            s = sigmoid(add)
            mulv = np.dot(V, s)
            layers.append({'s':s, 'prev_s':prev_s})
            prev_s = s

        # derivative of pred
        dmulv = (mulv - y)

        # backward pass
        for t in range(timestep):
            dV_t = np.dot(dmulv, np.transpose(layers[t]['s']))
            dsv = np.dot(np.transpose(V), dmulv)

            ds = dsv
            dadd = add * (1 - add) * ds

            dmulw = dadd * np.ones_like(mulw)

            dprev_s = np.dot(np.transpose(W), dmulw)

            for i in range(t-1, max(-1, t-bptt_truncate-1), -1):
                ds = dsv + dprev_s
                dadd = add * (1 - add) * ds

                dmulw = dadd * np.ones_like(mulw)
                dmulu = dadd * np.ones_like(mulu)

                dW_i = np.dot(W, layers[t]['prev_s'])
                dprev_s = np.dot(np.transpose(W), dmulw)

                new_input = np.zeros(x.shape)
                new_input[t] = x[t]
                dU_i = np.dot(U, new_input)
                dx = np.dot(np.transpose(U), dmulu)

                dU_t += dU_i
                dW_t += dW_i

            dV += dV_t
            dU += dU_t
            dW += dW_t

            if dU.max() > max_clip_value:
                dU[dU > max_clip_value] = max_clip_value
            if dV.max() > max_clip_value:
                dV[dV > max_clip_value] = max_clip_value
            if dW.max() > max_clip_value:
                dW[dW > max_clip_value] = max_clip_value


            if dU.min() < min_clip_value:
                dU[dU < min_clip_value] = min_clip_value
            if dV.min() < min_clip_value:
                dV[dV < min_clip_value] = min_clip_value
            if dW.min() < min_clip_value:
                dW[dW < min_clip_value] = min_clip_value

        # update
        U -= learning_rate * dU
        V -= learning_rate * dV
        W -= learning_rate * dW

        loss_per_record = (y - mulv)**2 / 2
        val_loss += loss_per_record

    val_loss = val_loss / float(y.shape[0])
    loss.append(val_loss[0][0])

In [62]:
loss

[6743.100886278886,
 13432.204639709826,
 20068.100203151153,
 26651.559546702316,
 33183.347167994354,
 39664.24067247593,
 46095.00677951323,
 52476.40069570372,
 58809.16633405147,
 65094.02744583083]